Ссылка на датасет: https://www.kaggle.com/c/dogs-vs-cats/data

# Dogs-vs-Cats Classifier by VGG16

## Import of Required Libs

In [80]:
import os
import random

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

## Data Preparation

In [36]:
path_to_files = '../datasets/transfer_learning/train'

images = [os.path.join(dp, f) for dp, dn, filenames 
          in os.walk(path_to_files) for f in filenames
          if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]

num_classes = 2

In [32]:
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x  

In [53]:
data = []
# Dogs assigned to 0, while cats to 1
cats_counter = 0
dogs_counter = 0
MAX_CAT_SIZE = 1500
for img_path in images:
    category = img_path.split('/')[-1].split('\\')[-1].split('.')[0]
    if category == 'dog' and dogs_counter < MAX_CAT_SIZE:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':0})
        dogs_counter += 1
    elif category == 'cat' and cats_counter < MAX_CAT_SIZE:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':1})
        cats_counter += 1
    elif dogs_counter == MAX_CAT_SIZE and cats_counter == MAX_CAT_SIZE:
        break

In [56]:
np.random.shuffle(data)
train_split = 0.8
idx_test = int((train_split) * len(data))

data_train = data[:idx_test]
data_test = data[idx_test:]

In [57]:
x_train, y_train = np.array([t["x"] for t in data_train]), [t["y"] for t in data_train]
x_test, y_test = np.array([t["x"] for t in data_test]), [t["y"] for t in data_test]

In [58]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# convert labels to one-hot vectors
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

## Basic VGG16 Model

In [61]:
vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

553467904/553467096 [==============================] - 196s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________________________________________________________

## VGG16 + New Classification Layer

In [62]:
# make a reference to VGG's input layer
inp = vgg.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(num_classes, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model = Model(inp, out)

In [63]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [64]:
for l, layer in enumerate(model.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
model.layers[-1].trainable = True

In [65]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [68]:
model.fit(x_train, y_train, batch_size=50, epochs=10)

Epoch 1/10
48/48 [==============================] - 281s 6s/step - loss: 0.3045 - accuracy: 0.8788
Epoch 2/10
48/48 [==============================] - 284s 6s/step - loss: 0.2869 - accuracy: 0.8913
Epoch 3/10
48/48 [==============================] - 295s 6s/step - loss: 0.2672 - accuracy: 0.8975
Epoch 4/10
48/48 [==============================] - 289s 6s/step - loss: 0.2573 - accuracy: 0.9008
Epoch 5/10
48/48 [==============================] - 275s 6s/step - loss: 0.2460 - accuracy: 0.9050
Epoch 6/10
48/48 [==============================] - 273s 6s/step - loss: 0.2469 - accuracy: 0.8967
Epoch 7/10
48/48 [==============================] - 269s 6s/step - loss: 0.2340 - accuracy: 0.9042
Epoch 8/10
48/48 [==============================] - 272s 6s/step - loss: 0.2338 - accuracy: 0.9104
Epoch 9/10
48/48 [==============================] - 268s 6s/step - loss: 0.2155 - accuracy: 0.9158
Epoch 10/10
48/48 [==============================] - 281s 6s/step - loss: 0.2190 - accuracy: 0.9117


**Wall time: 46 min**

In [70]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.21626615524291992
Test accuracy: 0.9116666913032532


**Final accuracy on test data: 91%**

In [75]:
# 2.jpg - dog picture
category = 0 
img, x = get_image('../datasets/transfer_learning/test/2.jpg')
probabilities = model.predict([x])
category == probabilities.argmax()

True

In [76]:
# 7.jpg - cat picture
category = 1
img, x = get_image('../datasets/transfer_learning/test/7.jpg')
probabilities = model.predict([x])
category == probabilities.argmax()

True

## Saving the model

In [79]:
model.save('../keras_models/vgg_model')

INFO:tensorflow:Assets written to: ../keras_models/vgg_model\assets
